In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.0 MB/s eta 0:00:00


In [11]:
import torch
import torch.nn.functional as F
import networkx as nx
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx, train_test_split_edges
from torch_geometric.nn import GCNConv, VGAE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [5]:
G = nx.read_graphml('airportsAndCoordAndPop.graphml')
print("Graph loaded successfully.")

Graph loaded successfully.


In [6]:
print("Le nombre de noeuds est : ",len(G.nodes))
print("Le nombre de liens est : ",len(G.edges))

Le nombre de noeuds est :  3363
Le nombre de liens est :  13547


In [7]:
 print("La densité du graphe est : ", nx.density(G))
 print("Le coefficient de clustering est : ", nx.transitivity(G))

La densité du graphe est :  0.002396340623182999
Le coefficient de clustering est :  0.24767449354443705


In [24]:
# Vérifier la connectivité
nx.is_connected(G)  # True ou False

False

In [25]:
components = list(nx.connected_components(G))
len(components)


6

In [23]:
for node, data in G.nodes(data=True):
    print(node, data)


0 {'lon': -145.50972222222222, 'lat': -17.35388888888889, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Anaa'}
1 {'lon': -140.95, 'lat': -18.066666666666666, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Hao Island'}
2 {'lon': -149.6, 'lat': -17.55, 'population': 26357, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Papeete'}
3 {'lon': -135.0, 'lat': -23.033333333333335, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Gambier Island'}
4 {'lon': -143.65725, 'lat': -16.584888888888887, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Makemo'}
5 {'lon': 174.79166666666666, 'lat': -37.00805555555556, 'population': 417910, 'country': 'NEW_ZEALAND', 'city_name': 'Auckland'}
6 {'lon': -139.03333333333333, 'lat': -9.8, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Atuona'}
7 {'lon': -151.75, 'lat': -16.45, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Bora Bora'}
8 {'lon': -157.9224

In [26]:
G.edges(data=True)


EdgeDataView([('0', '1', {}), ('0', '2', {}), ('1', '2', {}), ('1', '3', {}), ('1', '4', {}), ('2', '3', {}), ('2', '4', {}), ('2', '5', {}), ('2', '6', {}), ('2', '7', {}), ('2', '8', {}), ('2', '9', {}), ('2', '10', {}), ('2', '11', {}), ('2', '12', {}), ('2', '13', {}), ('2', '14', {}), ('2', '15', {}), ('2', '16', {}), ('2', '17', {}), ('2', '18', {}), ('2', '19', {}), ('2', '20', {}), ('2', '21', {}), ('2', '22', {}), ('2', '23', {}), ('2', '24', {}), ('2', '25', {}), ('2', '26', {}), ('2', '27', {}), ('5', '8', {}), ('5', '12', {}), ('5', '15', {}), ('5', '16', {}), ('5', '18', {}), ('5', '25', {}), ('5', '28', {}), ('5', '29', {}), ('5', '30', {}), ('5', '31', {}), ('5', '32', {}), ('5', '33', {}), ('5', '34', {}), ('5', '35', {}), ('5', '36', {}), ('5', '37', {}), ('5', '38', {}), ('5', '39', {}), ('5', '40', {}), ('5', '41', {}), ('5', '42', {}), ('5', '43', {}), ('5', '44', {}), ('5', '45', {}), ('5', '46', {}), ('5', '47', {}), ('5', '48', {}), ('5', '49', {}), ('5', '50', {

In [27]:
import plotly.graph_objects as go
edges = list(G.edges())

edge_x, edge_y = [], []
for u, v in edges:
    edge_x += [G.nodes[u]['lon'], G.nodes[v]['lon'], None]
    edge_y += [G.nodes[u]['lat'], G.nodes[v]['lat'], None]

node_x = [G.nodes[n]['lon'] for n in G.nodes]
node_y = [G.nodes[n]['lat'] for n in G.nodes]

fig = go.Figure()
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, mode='lines', line=dict(width=0.5), name='Edges'))
fig.add_trace(go.Scatter(x=node_x, y=node_y, mode='markers', marker=dict(size=6), name='Nodes'))
fig.show()


In [33]:

data = from_networkx(G, group_node_attrs=["lon", "lat", "population"])

# Affiche la structure
print(data)
print("Nombre de nœuds:", data.num_nodes)
print("Taille des features:", data.x.shape)
print("Features : \n", data.x)
print("Index des nœuds de départ et d'arrivée des arêtes : \n", data.edge_index)

Data(edge_index=[2, 27094], country=[3363], city_name=[3363], x=[3363, 3])
Nombre de nœuds: 3363
Taille des features: torch.Size([3363, 3])
Features : 
 tensor([[-1.4551e+02, -1.7354e+01,  1.0000e+04],
        [-1.4095e+02, -1.8067e+01,  1.0000e+04],
        [-1.4960e+02, -1.7550e+01,  2.6357e+04],
        ...,
        [-1.1320e+02,  5.4743e+01,  1.0000e+04],
        [ 1.4660e+02, -6.1333e+00,  1.0000e+04],
        [ 1.4725e+02, -6.0000e+00,  1.0000e+04]])
Index des nœuds de départ et d'arrivée des arêtes : 
 tensor([[   0,    0,    1,  ..., 3361, 3361, 3362],
        [   1,    2,    0,  ..., 3359, 3362, 3361]])


In [34]:
# Crée un split pour la prédiction de liens
data = train_test_split_edges(data, val_ratio=0.0, test_ratio=0.2)


/tmp/ipython-input-130846312.py:2: UserWarning:

'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead



In [35]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, VGAE

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2*out_channels)
        self.conv_mu = GCNConv(2*out_channels, out_channels)
        self.conv_logstd = GCNConv(2*out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)


In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channels = data.num_node_features
out_channels = 16

model = VGAE(Encoder(in_channels, out_channels)).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train_model(model, data, optimizer, epochs=200):
    model.train()
    for epoch in range(1, epochs+1):
        optimizer.zero_grad()
        z = model.encode(data.x, data.train_pos_edge_index)
        loss = model.recon_loss(z, data.train_pos_edge_index)
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            auc, ap = model.test(z, data.test_pos_edge_index, data.test_neg_edge_index)
            print(f"Epoch {epoch}, Loss: {loss:.4f}, AUC: {auc:.4f}, AP: {ap:.4f}")
    return model

model = train_model(model, data, optimizer, epochs=200)


Epoch 20, Loss: 3612610.2500, AUC: 0.5980, AP: 0.5550
Epoch 40, Loss: 789753.0625, AUC: 0.5901, AP: 0.5513
Epoch 60, Loss: 664947.5625, AUC: 0.5338, AP: 0.5180
Epoch 80, Loss: 652060.3750, AUC: 0.5124, AP: 0.5063
Epoch 100, Loss: 649830.8125, AUC: 0.5113, AP: 0.5058
Epoch 120, Loss: 649407.2500, AUC: 0.4994, AP: 0.4997
Epoch 140, Loss: 649356.3750, AUC: 0.4913, AP: 0.4957
Epoch 160, Loss: 649350.5625, AUC: 0.4959, AP: 0.4980
Epoch 180, Loss: 649349.4375, AUC: 0.5013, AP: 0.5006
Epoch 200, Loss: 649349.0000, AUC: 0.5015, AP: 0.5007


In [37]:
@torch.no_grad()
def evaluate_link_prediction(model, data):
    model.eval()
    z = model.encode(data.x, data.train_pos_edge_index)
    auc, ap = model.test(z, data.test_pos_edge_index, data.test_neg_edge_index)
    print(f"Final AUC: {auc:.4f}, AP: {ap:.4f}")
    return auc, ap

auc, ap = evaluate_link_prediction(model, data)


Final AUC: 0.7072, AP: 0.6320


In [42]:
# --- Reconstruction des probabilités de liens ---
with torch.no_grad():
    z = model.encode(data.x, data.train_pos_edge_index)
    adj_pred = torch.sigmoid(z @ z.T)

# --- Extraire les liens probables ---
edges_pred = []
for i in range(data.num_nodes):
    for j in range(i + 1, data.num_nodes):
        edges_pred.append((i, j, adj_pred[i, j].item()))

# Top 100 liens les plus probables
edges_pred = sorted(edges_pred, key=lambda x: x[2], reverse=True)[:100]

# --- Préparer les coordonnées pour Plotly ---
edge_x, edge_y = [], []
for i, j, _ in edges_pred:
    edge_x += [G.nodes[str(i)]['lon'], G.nodes[str(j)]['lon'], None]
    edge_y += [G.nodes[str(i)]['lat'], G.nodes[str(j)]['lat'], None]

node_x = [G.nodes[n]['lon'] for n in G.nodes]
node_y = [G.nodes[n]['lat'] for n in G.nodes]

# --- Créer les textes pour les nœuds ---
node_text = [
    f"City: {G.nodes[n]['city_name']}<br>Country: {G.nodes[n]['country']}<br>Population: {G.nodes[n]['population']}"
    for n in G.nodes
]

# --- Création de la figure interactive ---
fig = go.Figure()

# Arêtes prédites
fig.add_trace(go.Scatter(
    x=edge_x,
    y=edge_y,
    mode='lines',
    line=dict(width=1, color='red'),
    name='Predicted edges',
    hoverinfo='none'
))

# Nœuds
fig.add_trace(go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Airports',
    text=node_text,
    hoverinfo='text'
))

# --- Layout ---
fig.update_layout(
    title='Top 100 Predicted Links Between Airports (VGAE)',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    showlegend=True,
    width=900,
    height=600,
    hovermode='closest'
)

fig.show()
